In [1]:
import numpy as np
def calculate_distance(index1, index2):
    # 确保输入是字符串
    index1, index2 = str(index1), str(index2)

    # 解析点的index，得到单元号、行号、列号
    unit1, row1, col1 = int(index1[0]), int(index1[1]), int(index1[2])
    unit2, row2, col2 = int(index2[0]), int(index2[1]), int(index2[2])

    # 单元内的点之间的距离，行列的差加起来
    if unit1 == unit2:
        return abs(row1 - row2) + abs(col1 - col2)

    # 单元之间的距离，判断是否是相邻单元
    adjacent_units = {
        0: [1, 2],
        1: [0, 3],
        2: [0, 3],
        3: [1, 2]
    }

    # 边界判断逻辑
    def is_edge_point(unit, row, col):
        if unit == 0:
            return (row == 7 and 0 <= col <= 7) or (col == 7 and 0 <= row <= 7)  # Unit 0 边界
        elif unit == 1:
            return (row == 0 and 0 <= col <= 7) or (col == 7 and 0 <= row <= 7)  # Unit 1 边界
        elif unit == 2:
            return (row == 7 and 0 <= col <= 7) or (col == 0 and 0 <= row <= 7)  # Unit 2 边界
        elif unit == 3:
            return (row == 0 and 0 <= col <= 7) or (col == 0 and 0 <= row <= 7)  # Unit 3 边界
        return False

    # 边界点直接连接的判断（只有对应的边界点能直接连接）
    def can_connect_directly(unit1, row1, col1, unit2, row2, col2):
        if (unit1 == 0 and unit2 == 1) or (unit1 == 1 and unit2 == 0):
            return row1 == row2  # Unit 0 和 Unit 1 间的对应边界点直接连接
        elif (unit1 == 0 and unit2 == 2) or (unit1 == 2 and unit2 == 0):
            return col1 == col2  # Unit 0 和 Unit 2 间的对应边界点直接连接
        elif (unit1 == 1 and unit2 == 3) or (unit1 == 3 and unit2 == 1):
            return row1 == row2  # Unit 1 和 Unit 3 间的对应边界点直接连接
        elif (unit1 == 2 and unit2 == 3) or (unit1 == 3 and unit2 == 2):
            return col1 == col2  # Unit 2 和 Unit 3 间的对应边界点直接连接
        return False

    # 找到到边界点的最短距离
    def distance_to_edge(unit, row, col):
        if unit == 0:
            return min(7 - row, 7 - col)  # 单元 0
        elif unit == 1:
            return min(row, 7 - col)  # 单元 1
        elif unit == 2:
            return min(7 - row, col)  # 单元 2
        elif unit == 3:
            return min(row, col)  # 单元 3
        return np.inf

    # 处理相邻单元
    if unit2 in adjacent_units[unit1]:
        is_edge_point1 = is_edge_point(unit1, row1, col1)
        is_edge_point2 = is_edge_point(unit2, row2, col2)

        if is_edge_point1 and is_edge_point2:
            if can_connect_directly(unit1, row1, col1, unit2, row2, col2):
                # 如果两个边界点直接连接，距离为5
                return 5
            else:
                # 如果不能直接连接，计算通过边界点的路由距离
                dist_to_edge1 = distance_to_edge(unit1, row1, col1)
                dist_to_edge2 = distance_to_edge(unit2, row2, col2)
                return dist_to_edge1 + 5 + dist_to_edge2

    # 处理跨单元（不相邻单元）
    else:
        # 找到通过的中间单元
        if (unit1 == 0 and unit2 == 3) or (unit1 == 3 and unit2 == 0):
            mid_unit = 1  # 通过 unit1 进行路由
        elif (unit1 == 1 and unit2 == 2) or (unit1 == 2 and unit2 == 1):
            mid_unit = 0
        else:
            mid_unit = None
        
        if mid_unit is not None:
            # 计算点1到其边界点的距离
            dist_to_edge1 = distance_to_edge(unit1, row1, col1)
            # 计算点2到其边界点的距离
            dist_to_edge2 = distance_to_edge(unit2, row2, col2)
            # 计算通过中间单元的边界距离，边界到边界为5两次（跨中间单元）
            mid_distance = 10  # 中间单元跨越边界两次
            # 总路由距离为：点1到边界 + 跨中间单元边界距离 + 点2到边界
            return dist_to_edge1 + mid_distance + dist_to_edge2

    return np.inf





def create_distance_matrix():
    # 初始化256*256的距离矩阵
    _distance_matrix = np.full((256, 256), np.inf)

    for i in range(256):
        for j in range(256):
            # 生成index的字符串格式，例如"070", "101"
            index1 = f'{i // 64}{(i % 64) // 8}{i % 8}'
            index2 = f'{j // 64}{(j % 64) // 8}{j % 8}'
            
            # 计算距离
            _distance_matrix[i, j] = calculate_distance(index1, index2)
    
    return _distance_matrix

# 生成距离矩阵
distance_matrix = create_distance_matrix()

# 输出部分矩阵结果来查看
print(distance_matrix[:40, :40])  # 打印前10行和前10列的距离


[[ 0.  1.  2. ...  9. 10. 11.]
 [ 1.  0.  1. ...  8.  9. 10.]
 [ 2.  1.  0. ...  7.  8.  9.]
 ...
 [ 9.  8.  7. ...  0.  1.  2.]
 [10.  9.  8. ...  1.  0.  1.]
 [11. 10.  9. ...  2.  1.  0.]]


In [2]:
# 修改程序，将结果输出为一个.txt文档

def save_distance_matrix_to_txt():
    # 生成距离矩阵
    distance_matrix = create_distance_matrix()
    
    # 保存为txt文件
    np.savetxt("distance_matrix.txt", distance_matrix, fmt="%.2f")
    
    return "distance_matrix.txt"

# 保存为txt文件
output_txt_file = save_distance_matrix_to_txt()
output_txt_file


'distance_matrix.txt'

In [3]:
calculate_distance("000", "377")

24